# Import

In [3]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn import datasets
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict


from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve
from scipy.stats import randint as sp_randint


In [89]:
import re

# Data Load

In [60]:
path = os.getcwd() + '\\new-york-city-census-data\\nyc_census_tracts.csv'
data = pd.read_csv(path) #load data into dataframe
#data

data.head(n=7)

,CensusTract,County,Borough,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,36005000100,Bronx,Bronx,7703,7133,570,29.9,6.1,60.9,0.2,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,36005000200,Bronx,Bronx,5403,2659,2744,75.8,2.3,16.0,0.0,...,2.9,0.0,0.0,43.0,2308,80.8,16.2,2.9,0.0,7.7
2,36005000400,Bronx,Bronx,5915,2896,3019,62.7,3.6,30.7,0.0,...,1.4,0.5,2.1,45.0,2675,71.7,25.3,2.5,0.6,9.5
3,36005001600,Bronx,Bronx,5879,2558,3321,65.1,1.6,32.4,0.0,...,8.6,1.6,1.7,38.8,2120,75.0,21.3,3.8,0.0,8.7
4,36005001900,Bronx,Bronx,2591,1206,1385,55.4,9.0,29.0,0.0,...,3.0,2.4,6.2,45.4,1083,76.8,15.5,7.7,0.0,19.2
5,36005002000,Bronx,Bronx,8516,3301,5215,61.1,1.6,31.1,0.3,...,4.3,1.0,0.0,46.0,2508,71.0,21.3,7.7,0.0,17.2
6,36005002300,Bronx,Bronx,4774,2130,2644,62.3,0.2,36.5,1.0,...,14.0,1.5,4.1,42.7,1191,74.2,16.1,9.7,0.0,18.9


In [61]:
from sklearn.svm import SVR

In [62]:
data.shape

(2167, 36)

In [63]:
dummy_col_1 = np.zeros(data.shape[0])
dummy_col_2 = np.ones(data.shape[0])
dummy_col_3 = ['NY'] * data.shape[0]
dummy_col_1,dummy_col_2,dummy_col_3 = pd.DataFrame(dummy_col_1),pd.DataFrame(dummy_col_2),pd.DataFrame(dummy_col_3)
dummy_col_1.head(n = 7)

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0


In [64]:
data['Num_Arrests'] = dummy_col_2
data['State'] = dummy_col_3
data['Race'] = dummy_col_1

In [65]:
data.shape

(2167, 39)

In [80]:
data_train = data.sample(frac=0.7,random_state=200) #meant to randomly grab 70% of data by row for test
data_test  = data.drop(data_train.index)

In [81]:
cols = data.shape[1] #num columns

X = data_train.iloc[:,0:cols-1] # iloc slicing function , split the data into X and Y
y = data_train.iloc[:,cols-1:cols]


In [82]:
X.columns

Index(['CensusTract', 'County', 'Borough', 'TotalPop', 'Men', 'Women',
       'Hispanic', 'White', 'Black', 'Native', 'Asian', 'Citizen', 'Income',
       'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment', 'Num_Arrests', 'State'],
      dtype='object')

In [83]:
def model_fit(my_kernel,X,y):
    clf = svm.SVR(kernel = my_kernel, gamma='auto')
    clf.fit(X,y)
    print('================'+str(my_kernel)+'==================')
    cross_val(clf, X, y)

In [84]:
def cross_val(clf, X, y ):
    CV = KFold(n_splits=50, random_state=None, shuffle=False)
    y_pred = cross_val_predict(clf, X, y, cv = CV)
    print('Classification Report: \n')
    print(metrics.classification_report(y, y_pred, sample_weight = y,labels=np.unique(y_pred)))
    print('\nConfusion Matrix: ')
    print(metrics.confusion_matrix(y, y_pred, sample_weight = y))

# DONT RUN THIS ONE UNTIL READY

In [ ]:
kernals = ['linear', 'poly', 'rbf', 'sigmoid']
CV = 10
for kernal in kernals:
    model_fit(kernal,X,y)

# Data Loading 
## Census Block Loc

In [85]:
path = os.getcwd() + '\\new-york-city-census-data\\census_block_loc.csv'
data_block = pd.read_csv(path) #load data into dataframe
#data

data_block.head(n=7)

,Latitude,Longitude,BlockCode,County,State
0,40.48,-74.280000,340230076002012,Middlesex,NJ
1,40.48,-74.276834,340230076005000,Middlesex,NJ
2,40.48,-74.273668,340230076003018,Middlesex,NJ
3,40.48,-74.270503,340230076003004,Middlesex,NJ
4,40.48,-74.267337,340230074021000,Middlesex,NJ
5,40.48,-74.264171,340230074021000,Middlesex,NJ
6,40.48,-74.261005,340230074021000,Middlesex,NJ


# Data Loading
## NYC Arrest Data

In [86]:
path = os.getcwd() + '\\NYPD_Arrest_Data__Year_to_Date_.csv'
data_arrest = pd.read_csv(path) #load data into dataframe
#data

data_arrest.head(n=7)

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
0,199131647,06/30/2019,109.0,"ASSAULT 2,1,UNCLASSIFIED",106.0,FELONY ASSAULT,PL 1200501,F,K,78,0,<18,M,BLACK,988709,187224,40.680570,-73.983925
1,199109946,06/30/2019,244.0,"BURGLARY,UNCLASSIFIED,UNKNOWN TIME",107.0,BURGLARY,PL 1402000,F,Q,104,0,25-44,M,BLACK,1011210,191981,40.693587,-73.902781
2,199131650,06/30/2019,792.0,WEAPONS POSSESSION 1 & 2,118.0,DANGEROUS WEAPONS,PL 2650303,F,K,73,0,18-24,M,BLACK,1007783,185486,40.675770,-73.915162
3,199126329,06/30/2019,922.0,"TRAFFIC,UNCLASSIFIED MISDEMEANOR",348.0,VEHICLE AND TRAFFIC LAWS,VTL05110A2,M,Q,104,0,25-44,M,WHITE HISPANIC,1017324,198914,40.712596,-73.880699
4,199127207,06/30/2019,339.0,"LARCENY,PETIT FROM OPEN AREAS,UNCLASSIFIED",341.0,PETIT LARCENY,PL 1552500,M,K,90,0,45-64,M,WHITE HISPANIC,999701,195491,40.703249,-73.944275
5,199127210,06/30/2019,779.0,"PUBLIC ADMINISTRATION,UNCLASSIFIED FELONY",126.0,MISCELLANEOUS PENAL LAW,PL 215510B,F,K,75,0,45-64,F,WHITE,1019699,188374,40.683657,-73.872188
6,199128426,06/30/2019,478.0,"THEFT OF SERVICES, UNCLASSIFIED",343.0,OTHER OFFENSES RELATED TO THEFT,PL 1651502,M,M,14,0,25-44,F,BLACK,987472,214939,40.756642,-73.988372


In [87]:
X['Non-White'] = 100 - X['White']

In [88]:
X.drop(columns=['Hispanic','Black','Native','Asian'])

,CensusTract,County,Borough,TotalPop,Men,Women,White,Citizen,Income,IncomeErr,...,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,Num_Arrests,State,Non-White
296,36005040501,Bronx,Bronx,4416,2088,2328,4.2,2137,34009.0,7616.0,...,40.5,1883,82.5,9.1,8.4,0.0,12.9,1.0,NY,95.8
192,36005026500,Bronx,Bronx,7009,3116,3893,2.0,3393,31147.0,4465.0,...,43.3,3173,86.8,7.8,5.4,0.0,13.0,1.0,NY,98.0
1096,36047123700,Kings,Brooklyn,6522,3520,3002,83.6,2417,30035.0,5604.0,...,28.1,1890,93.9,2.9,3.3,0.0,4.7,1.0,NY,16.4
701,36047042300,Kings,Brooklyn,4986,2803,2183,16.1,2327,51083.0,24847.0,...,37.7,2442,85.3,8.6,6.1,0.0,12.8,1.0,NY,83.9
1777,36081058500,Queens,Queens,4476,2117,2359,40.0,2646,52917.0,9121.0,...,38.0,1866,88.5,9.3,2.0,0.3,7.0,1.0,NY,60.0
143,36005022000,Bronx,Bronx,2176,1172,1004,0.3,599,23167.0,5153.0,...,42.0,842,89.7,4.8,5.6,0.0,19.5,1.0,NY,99.7
407,36047009600,Kings,Brooklyn,5951,3067,2884,18.4,2321,32829.0,6804.0,...,39.8,2461,81.1,6.9,11.9,0.0,10.9,1.0,NY,81.6
826,36047056600,Kings,Brooklyn,2747,1315,1432,53.0,1874,61779.0,12131.0,...,50.2,1233,74.9,21.5,2.9,0.6,5.9,1.0,NY,47.0
719,36047044100,Kings,Brooklyn,6156,3308,2848,11.6,2653,49167.0,7311.0,...,38.4,3215,90.9,3.3,5.8,0.0,12.0,1.0,NY,88.4
1881,36081077902,Queens,Queens,5382,2692,2690,84.6,3324,83629.0,16022.0,...,37.9,2005,79.0,10.1,10.9,0.0,5.7,1.0,NY,15.4
